In [1]:
import base64
import urllib
import urllib.request
import cv2
from multiprocessing import Process, Queue
import os
from IPython import display
from time import sleep

access_token = '24.3a8df80dc63ce8ac24ec1a7b115f2d5f.2592000.1574820093.282335-17330240'
#__VideoIndex__ = './Data/WIN_20190927_12_48_37_Pro.mp4'
#__VideoIndex__ = './NEW_DATA/1.mp4'
__VideoIndex__ = None

In [2]:
import sys
import json
import time
import pyaudio
import wave

IS_PY3 = sys.version_info.major == 3

if IS_PY3:
    from urllib.request import urlopen
    from urllib.request import Request
    from urllib.error import URLError
    from urllib.parse import urlencode
    timer = time.perf_counter
else:
    from urllib2 import urlopen
    from urllib2 import Request
    from urllib2 import URLError
    from urllib import urlencode
    if sys.platform == "win32":
        timer = time.clock
    else:
        # On most other platforms the best timer is time.time()
        timer = time.time

In [3]:
'''
图像预处理
'''
def ImgPreprocess(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    (b,g,r) = cv2.split(frame)
    #b = cv2.equalizeHist(b)
    #g = cv2.equalizeHist(g)
    r = cv2.equalizeHist(r)
    frame = cv2.merge((b,g,r))
    frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
    return frame


'''
人体关键点识别
'''
def KeyPointDetect(img_base64):
    request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/body_analysis"
    params = {"image":img_base64}

    params = urllib.parse.urlencode(params)
    request_url = request_url + "?access_token=" + access_token

    params = params.encode('utf-8')
    header = {'Content-Type': 'application/x-www-form-urlencoded'}
    request = urllib.request.Request(url=request_url, data=params, headers = header)

    response = urllib.request.urlopen(request)
    content = response.read()

    if content:
        return content
    else:
        return None

'''
人脸检测与属性分析
'''
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def myFaceDetect(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))#, flags=cv2.CASCADE_SCALE_IMAGE)
    face_num = len(faces)
    print('Detected ', len(faces), ' face!')
    return face_num

face_trig_list = [];
last_face_valid_time = 0
def myFaceTrig(cur_face):
    global face_trig_list
    global last_face_valid_time
    
    result = 0
    cur_time = time.time()
    
    if cur_face == 0:
        cur_trig = 0
    else:
        cur_trig = 1
    
    face_trig_list = face_trig_list + [cur_trig]
    
    if len(face_trig_list) > 10:
        face_trig_list = face_trig_list[-10:]
        
    if len(face_trig_list) == 10:
        if face_trig_list.count(1) >6:
            result = 1
            last_face_valid_time = cur_time
    
    if cur_time - last_face_valid_time < 10:
        result = 1
    
    
    return result
    

def FaceDetect(img_base64):
    # encoding:utf-8

    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/detect"
    
    params = {"image":img_base64,"image_type":"BASE64","face_field":"faceshape,facetype"}
    params = urllib.parse.urlencode(params)
    
    params = params.encode('utf-8')
    request_url = request_url + "?access_token=" + access_token
    header = {'Content-Type': 'application/json'}
    request = urllib.request.Request(url=request_url, data=params, headers = header)

    response = urllib.request.urlopen(request)
    content = response.read()
    if content:
        #content = eval(str(content, encoding = "utf-8"))
        return content
    else:
        return None

    
def ExtractFacePoint(res, frame, face_index = 0, graph = False):
    
    face_location = res['result']['face_list'][face_index]['location']
    face_lefttop = (int(face_location['left']),int(face_location['top']))
    face_rightdown = (int(face_location['left'] + face_location['width']), int(face_location['top'] + face_location['height']))
    
    if graph == True:
        cv2.rectangle(frame,face_lefttop, face_rightdown,(255,255,255),1,4)
    
    return (face_lefttop, face_rightdown, frame)
    
    
'''
配置视频输入流
'''
def InitVideo():
    global capture

    if __VideoIndex__:
        capture = cv2.VideoCapture(__VideoIndex__)
    else:
        capture = cv2.VideoCapture(2)

    if capture.isOpened():
        return True
    else:
        raise Exception("camera failed")

'''
释放视频输入设备
'''
def ReleaseVideo():
    capture.release()

'''
调用摄像头返回图像
'''
def ReadImg():
    ref,frame = capture.read()
    return ref,frame
    
def SaveImg(frame):
    cv2.imwrite('temp_0.jpg',frame)
    
'''
np图像转base64编码
'''
def Img2Base64(img):
    img_temp = cv2.imencode('.jpg',img)[1]
    img_base64 = base64.b64encode(img_temp)
    img_base64 = str(img_base64, encoding='utf-8')
    return img_base64

def LoadImgFile():
    with open('temp_0.jpg','rb') as f:
        img = base64.b64encode(f.read())
    return img


#internal trig function
def _TrigGen(body_parts):
    if body_parts:
        #print('left w - left s = ',abs(body_parts['left_wrist']['y'] - body_parts['left_shoulder']['y']))
        #print('right w - right s = ' , abs(body_parts['right_wrist']['y'] - body_parts['right_shoulder']['y']))
        
        if abs(body_parts['left_wrist']['y'] - body_parts['left_shoulder']['y']) < 30 or \
            abs(body_parts['right_wrist']['y'] - body_parts['right_shoulder']['y']) < 30:
            return 1
        
        return 0
    else:
        raise Exception("content is None")



# trigger the latter function.
trigger_list = []# direct trig res
trigger_res_list = []
trig_frame_wait = 0
asr_keyword_flag = 0
asr_valid_flag = 0
def Trigger(body_parts,face_num):
    result = 0
    global trigger_list
    global trigger_res_list
    global trig_frame_wait
    
    asr_keyword_flag = Asr_trig
    asr_valid_flag = Asr_valid
    
    LIST_INDEX_MAX = 5
    RES_INDEX_MAX = 10
    
    asr_wait = 0
    
    if not trig_frame_wait:

        
        cur_res = 0
        cur_trig = _TrigGen(body_parts)
        trigger_list = trigger_list + [cur_trig]
        sum_list_index = 0
        sum_trig_index = 0
        
        for i in range(2):#0-1
            sum_list_index = sum_list_index + trigger_list.count(i)
        if sum_list_index > LIST_INDEX_MAX: #
            trigger_list = trigger_list[-LIST_INDEX_MAX:]
            sum_list_index = LIST_INDEX_MAX
            
        for i in range(3):#0-1
            sum_trig_index = sum_trig_index + trigger_res_list.count(i)
        if sum_trig_index > RES_INDEX_MAX: #
            trigger_res_list = trigger_res_list[-RES_INDEX_MAX:]
            sum_trig_index = RES_INDEX_MAX
            
        if sum_list_index == LIST_INDEX_MAX:
            if trigger_list.count(1) > 3:
                cur_res = 1
            if trigger_list.count(0) > 3:
                cur_res = 2
            
            trigger_res_list = trigger_res_list + [cur_res]
        
        if face_num > 0 and asr_wait == 0 and (sum_list_index == LIST_INDEX_MAX and sum_trig_index == RES_INDEX_MAX):
            if trigger_res_list[:6].count(1) > 2 and trigger_res_list[-5:].count(2) > 3 or\
            trigger_res_list[:6].count(1) > 3 and trigger_res_list[-5:].count(2) > 1:
                
                if asr_valid_flag == 0:
                    result = 1
                    trig_frame_wait = 10
                    trigger_list = []# direct trig res
                    trigger_res_list = []
                
                else:
                    asr_wait = 1
                    
        if asr_wait == 1:
            if asr_valid_flag == 0:
                result = 1
                asr_wait = 0
                
    else:
        trig_frame_wait = trig_frame_wait - 1
    print(trigger_list)
    print(trigger_res_list)
    return result
        
    
        
'''
服务器申请函数
'''
def ImgService(frame):
    SaveImg(frame)
    img_base64 = LoadImgFile()
    res = KeyPointDetect(img_base64)
    return res
        
'''
图像处理主函数，测试用
'''
def ImgProcess():
    InitVideo()
    for i in range(50):
        frame = ReadImg()
        frame = cv2.resize(frame,(640,480),interpolation=cv2.INTER_CUBIC)
        frame = ImgPreprocess(frame)
        res = ImgService(frame)
        #res = None
        if res:
            res = eval(str(res, encoding = "utf-8"))
            #print(res)

            body_parts = {'left_wrist':0,'right_wrist':0, 'left_shoulder':0, 'right_shoulder':0}
            if 'person_num' in res:
                if res['person_num'] == 1:        
                    for i in body_parts:
                        body_parts[i] = res['person_info'][0]['body_parts'][i]
                    #print("\r",body_parts,end="",flush=True)
                    for i in body_parts:
                        if body_parts[i]['score']>0.5:
                            cv2.circle(frame,(int(body_parts[i]['x']),int(body_parts[i]['y'])),10,(0,255,255),4)
            
            #人脸使用
            #if res['error_code'] == 0:

                #KeyPointRes = KeyPointDetect(frame)
                #print(1)

        cv2.imshow("1",frame)
        cv2.waitKey(1)

    cv2.destroyAllWindows()
    ReleaseVideo()
    
    
    
def PutQueue(img_q,img):
    if img_q.qsize()<500:
        img_q.put(img)
    else:
        img_q.get(True)
        img_q.put(img)
        
def OutputMov(img_q,rcd_index):
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 保存视频的编码
    

    out = cv2.VideoWriter('./temp/output'+str(rcd_index)+'.mp4',fourcc, 30.0, (640,480))
    #while not img_q.empty():
    while img_q.qsize() > 0:
        frame = img_q.get(True)
        out.write(frame)
    out.release()
    if rcd_index == 0:
        os.system('ffmpeg -i ./temp/output' + str(rcd_index) +'.mp4 -vcodec h264 ./HTML/video.mp4')
    else:
        os.system('ffmpeg -i ./temp/output' + str(rcd_index) +'.mp4 -vcodec h264 ./HTML/video' + str(rcd_index) + '.mp4')
        
    
'''
多进程
serv_flag_q 是 图像读入后 通知serv开始执行的符号
当serv成功返还res后 serv_p 将flag置空
当检测到servflag 非空时，图像线程将直接进行打印上次结果
'''

def img_p(frame_q,serv_flag_q,res_q,stop_q,img_q1,img_q2,save_q,stop2_q):
    global asr_keyword_flag
    
    QUEUE_IN_USE = 1
    RECORD_INDEX = 0
    
    InitVideo()
    
    X_upload = 320
    Y_upload = 240
    X_saveh263 = 640#352
    Y_saveh263 = 480#288
    body_parts =  {'left_wrist': {'y': 1, 'x': 1, 'score': 0}, 'right_wrist': {'y': 1, 'x': 1, 'score': 0}, 'left_shoulder': {'y': 1, 'x': 1, 'score': 0}, 'right_shoulder': {'y': 1, 'x': 1, 'score': 0}}#init body part
    ocr_index = 0
    last_trig = 0
    for i in range(2700):
        ref,frame_orig = ReadImg()
        if ref == False:
            break
        frame_save = cv2.resize(frame_orig,(X_saveh263,Y_saveh263),interpolation=cv2.INTER_CUBIC)
        
        
        #frame_save = frame_orig
        
        
        if (QUEUE_IN_USE == 1): # if keyword detected, refresh the buffer
            PutQueue(img_q1,frame_save)
            if Asr_trig == 1 and last_trig == 0:
                while img_q1.qsize()>0 :
                    img_q1.get(True)
        else:
            PutQueue(img_q2,frame_save)
            if Asr_trig == 1 and last_trig == 0:
                while img_q2.qsize()>0 :
                    img_q2.get(True)
        last_trig = Asr_trig
        
        frame = cv2.resize(frame_orig,(X_upload,Y_upload),interpolation=cv2.INTER_CUBIC)
        frame = ImgPreprocess(frame)
        
        face_num = myFaceDetect(frame)
        face_trig = myFaceTrig(face_num)
        
       
        
        res = 0 
        if serv_flag_q.qsize() == 0:#如果服务器闲 serv_q is empty
            #print('serv_p is idle')
            if res_q.qsize() > 0:#如果服务器有返回的res,res_q is not empty
                res = res_q.get(True)
            #print('write a frame')
            frame_q.put(frame)
            #print('start a serv')
            serv_flag_q.put(1)#置服务器忙状态
            
        if res:
            
            res = eval(str(res, encoding = "utf-8"))
            #print(res)
            
            
            if 'person_num' in res:
                if res['person_num'] >0:        
                    for i in body_parts:
                        body_parts[i] = res['person_info'][0]['body_parts'][i]
                    #print("\r",body_parts,end="",flush=True)
            
            trig = Trigger(body_parts,face_trig)
            print("trigger = ",trig)
            if trig:#
                save_q.put(QUEUE_IN_USE)
                cv2.imwrite('./HTML/'+str(0)+'.jpg',frame_orig)
                ocr_index = ocr_index + 1
                
                if QUEUE_IN_USE == 1:
                    QUEUE_IN_USE = 2
                else:
                    QUEUE_IN_USE = 1
                RECORD_INDEX = RECORD_INDEX + 1
            
            
            #人脸使用
            #if res['error_code'] == 0:

                #KeyPointRes = KeyPointDetect(frame)
                #print(1)

        
        # print body_parts
        for i in body_parts:
            if body_parts[i]['score']>0.3:
                cv2.circle(frame_orig,(int(body_parts[i]['x']/X_upload*frame_orig.shape[1]),int(body_parts[i]['y']/Y_upload*frame_orig.shape[0])),10,(0,255,255),4)
        #print(body_parts)
        
        cv2.imshow("1",frame_orig)
        cv2.waitKey(1)
        
    
    
    print('start stopping')


    stop_q.put(1)
    stop2_q.put(1)
    
    
    sleep(5)
    os.system('python OCR.py')
    while img_q1.qsize()>0:
        img_q1.get(True)
    while img_q2.qsize()>0:
        img_q2.get(True)
    cv2.destroyAllWindows()
    ReleaseVideo()
    
    

def serv_p(frame_q,serv_flag_q,res_q,stop_q):
    while(stop_q.qsize()==0):

        #display.clear_output(wait = True)
        if not serv_flag_q.qsize()>0:#如果置位了服务器忙信号 serv flag is not empty
            #if frame_q.qsize()==0:#如果此时frame队列没有东西，则应该是脑子出了问题
                #raise Exception('Frame is empty')

            frame = frame_q.get(True)#提取图像
            print('get a frame')
            res = ImgService(frame)#提交服务器
            if res_q.qsize()>0:
                raise Exception('Res is not empty')
            res_q.put(res)
            #print('stop a serv')
            serv_flag_q.get(True)#取消服务器忙信号

    print('serv_p_stopped')


def save_p(img_q1,img_q2, save_q, stop2_q):
    rcd_index = 0
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 保存视频的编码
    while(stop2_q.qsize()==0):
        if save_q.qsize() > 0:# if need to save  
            save_q.put(1)
            queue_in_use = save_q.get(True)
            if queue_in_use == 1:
                OutputMov(img_q1,rcd_index)
            else:
                OutputMov(img_q2,rcd_index)
            rcd_index = rcd_index + 1
            save_q.get(True) # reset the save flag
            print('saved')
    stop2_q.get(True)
    print('save_p stopped')
    


In [4]:
API_KEY = 'kVcnfD9iW2XVZSMaLMrtLYIz'
SECRET_KEY = 'O9o1O213UgG5LFn0bDGNtoRN3VWl2du6'

# 普通版
#DEV_PID = 1536
#ASR_URL = 'http://vop.baidu.com/server_api'
# 极速版
DEV_PID = 80001
ASR_URL = 'https://vop.baidu.com/pro_api'

token = '24.9d7d643527ee67f41ed64f11e5b58444.2592000.1573186085.282335-15803531'

CHUNK = 16000
RATE = 16000
RECORD_SECONDS = 90

params = {'dev_pid': DEV_PID,
          'format': 'pcm',
          'rate': RATE,
          'token': token,
          'cuid': 'hmi-test',
          'channel': 1,
          'speech': 'none',
          'len': 0
          }

BACK_NOISE = 1000
SPEECH_LEVEL = 20000
KWS = ['记住', '记下来', '重点', '重要', '要考', '要提问']

In [5]:
Asr_valid = False
Asr_trig  = False
frames_int_pool = []

In [6]:
def raw_to_int(raw_data, lens=0):
    speech_data_ints = []
    data_sum = 0
    if lens==0:
        lens = len(raw_data)
    for i in range(0, lens, 2):
        speech_data_int = int.from_bytes(raw_data[i:i+2], byteorder='little', signed=True)
        speech_data_ints.append(speech_data_int)
        if abs(speech_data_int) > BACK_NOISE:
            data_sum += abs(speech_data_int)
    return speech_data_ints, data_sum

In [7]:
def recording(speech_datas, valid_q):
    global Asr_valid, frames_int_pool
    frames = []
    frames_int = []
    frames_sum = []

    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    print("开始录音，请说话......")

    # 先录音4s
    for i in range(4):
        data = stream.read(CHUNK)
        frames.append(data)
        data_int, data_sum = raw_to_int(data)
        frames_int.append(data_int)
        frames_sum.append(data_sum)
    print('录音启动完成')

    # 1s步长，5s窗口
    for i in range(0, RECORD_SECONDS):
        #print('\ni =', i)
        data = stream.read(CHUNK)
        frames.append(data)
        data_int, data_sum = raw_to_int(data)
        frames_int.append(data_int)
        frames_sum.append(data_sum)
        # 窗口内强度＞阈值，则认为有语音
        #print(frames_sum)
        if sum(frames_sum) > SPEECH_LEVEL:
            Asr_valid = True
            valid_q.put(True)
            #print('speech')
        else:
            Asr_valid = False
            valid_q.put(False)
            #print('silence')
        # 把窗口内的语音放入队列
        speech_datas.put(b''.join(frames))
        frames.pop(0)
        frames_sum.pop(0)

    speech_datas.put('Done')
    Asr_valid = False
    #print('录音完成，请闭嘴！')
    # 获取原始数据，用于调试
    #frames_int_pool = [i for item in frames_int for i in item]
    
    stream.stop_stream()
    stream.close()
    p.terminate()

In [8]:
def speech_process(speech_datas, valid_q):
    global Asr_valid, Asr_trig
    j = 0
    speech_data = 0
    while True:
        speech_data = speech_datas.get(True)
        #display.clear_output(wait=True)
        if speech_data == 'Done':
            print('\nOVER......')
            Asr_trig = False
            break
        print('\nj =', j)
        j += 1
        if not valid_q.get():
            print('Silience')
            Asr_trig = False
            continue
        params['speech'] = str(base64.b64encode(speech_data), 'utf-8')
        params['len'] = len(speech_data)
        post_data = json.dumps(params, sort_keys=False)
        req = Request(ASR_URL, post_data.encode('utf-8'))
        req.add_header('Content-Type', 'application/json')

        try:
            begin = timer()
            f = urlopen(req)
            result_str = str(f.read(), 'utf-8')
            print ("Request time cost %f" % (timer() - begin))
        except URLError as err:
            #print('asr http response http code : ' + str(err.code))
            #result_str = str(err.read(), 'utf-8')
            Asr_trig = False
            continue
        result = json.loads(result_str)

        if result['err_no'] == 0:
            s = result['result'][0]
            print(s)
            for i,key in enumerate(KWS):
                if key in s:
                    Asr_trig = True
                    print('trig:', key)
                    break
                elif i==len(KWS)-1:
                    Asr_trig = False
        else:
            print(result)
            Asr_trig = False

In [9]:
f_q = Queue()
s_q = Queue()
r_q = Queue()
st_q = Queue()
st2_q = Queue()
img_q1 = Queue()
img_q2 = Queue()
save_q = Queue()


speech_datas = Queue()
valid_q = Queue()
pw = Process(target=recording, args=(speech_datas,valid_q))
pr = Process(target=speech_process, args=(speech_datas,valid_q))

pi = Process(target = img_p, args = (f_q,s_q,r_q,st_q,img_q1,img_q2,save_q,st2_q,))
ps = Process(target = serv_p, args = (f_q,s_q,r_q,st_q,))
psv = Process(target = save_p, args = (img_q1,img_q2,save_q,st2_q,))
pi.start()
ps.start()
psv.start()
# 启动子进程pw，写入:
pw.start()
# 启动子进程pr，读取:
pr.start()
# 等待pw结束，pr会自动结束:



#pw.join()
pi.join()

开始录音，请说话......
Detected  1  face!
get a frame
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
[0]
[]
get a frame
trigger =  0
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
[0, 0]
get a frame
[]
trigger =  0
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
[0, 0, 0]
get a frame
[]
trigger =  0
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  face!
Detected  1  

In [ ]:
ReleaseVideo()

In [ ]:
body_parts = {'left_wrist':0,'right_wrist':0, 'left_shoulder':0, 'right_shoulder':0}
if 'person_num' in res:
    if res['person_num'] == 1:        
        for i in body_parts:
            body_parts[i] = res_1['person_info'][0]['body_parts'][i]
        
        for i in body_parts:
            if body_parts[i]['score']>0.7:
                cv2.circle(frame,(body_parts[i]['y'],(body_parts[i]['x']),10,(0,255,255),4)

In [ ]:
cam = cv2.VideoCapture(0)

In [ ]:
ret, frame = cam.read()

In [ ]:
face_cascade

In [ ]:
myFaceDetect(frame)

In [ ]:

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
trigger_res_list[:6].count(1) > 4 and trigger_res_list[-5:].count(2) > 3

In [ ]:
cv2.destroyAllWindows()

In [ ]:
from time import time

In [ ]:
time()

In [ ]:
sleep(2)

In [ ]:
import os

In [ ]:
os.system('ffmpeg -i outeeput' + str(0) +'.mp4 -vcodec h264 ' + str(123) + '.mp4')

In [ ]:
import cv2
cam = cv2.VideoCapture(2)
ret,frame = cam.read()
cv2.imshow('asd',frame)
cv2.waitKey(0)
cam.release()

In [ ]:
cam = cv2.VideoCapture(1)

In [ ]:
cv2.imshow('asd',frame)
cv2.waitKey(0)

In [ ]:
from smb.SMBConnection import SMBConnection
host = '192.168.188.102'
username = 'MALIC'
password = '123456++'
conn = SMBConnection(username, password,"","",use_ntlm_v2=True)
result = conn.connect(host,445)
result

In [ ]:
cam.release()

In [ ]:
cv2.destroyAllWindows()